In [326]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
import pandas as pd
import numpy as np
import os
from sqlalchemy import text
import plotly.express as px
import plotly.graph_objects as go
from itertools import chain



In [327]:
def retorna_venda_item():

    call = (text(""""""))

    return call
                   

In [328]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os
from pathlib import Path


myboxhost_mssql= os.getenv()
myboxuser_mssql= os.getenv()
myboxdatabase_mssql= os.getenv()
myboxpassword_mssql= os.getenv()



def mssql_get_conn():

    connection_url = URL.create(
            "mssql+pyodbc",
            username="myboxhost_mssql",
            password="myboxpassword_mssql",
            host="myboxhost_mssql",
            database="myboxdatabase_mssql",
            query={
                "driver": "ODBC Driver 17 for SQL Server",
                "autocommit": "True",
        },
        )
      
    engine = create_engine(connection_url).execution_options(
    isolation_level="AUTOCOMMIT", future=True,fast_executemany=True)
    return engine

In [329]:
def get_ambientados():
    enginemssql = mssql_get_conn()
    with enginemssql.begin() as conn:
        items = retorna_venda_item()
 
        get_items = conn.execute(items).all()
        dict_tems = [row._asdict() for row in get_items]
        return dict_tems
    
testes = get_ambientados()


In [ ]:
data = pd.DataFrame(testes)
data.dropna()


data.columns
precos_df = data[['vendaId', 'produtoId', 'quantidade', 'valorUnitario', 'total',
       'dataCadastro', 'valorUnitarioCusto', 'valorFrete','nomeunidade']]

precos_df['dataCadastro'] = precos_df['dataCadastro'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))
precos_df['produtoId'] = precos_df['produtoId'].astype(int)

contagem = precos_df['produtoId'].value_counts().reset_index()
contagem


merged = pd.merge(precos_df,contagem, left_on='produtoId', right_on='index', how='left')

merged.rename(columns={'produtoId_y': 'contagem'}, inplace=True)


precos_df = merged[['vendaId', 'produtoId_x', 'quantidade', 'valorUnitario', 'total',
       'dataCadastro', 'valorUnitarioCusto', 'valorFrete','contagem','nomeunidade']]
precos_df = precos_df.drop_duplicates()

In [331]:
(np.random.random() * 0.5) * 100

17.945835802795816

In [332]:
def add_discount_data(df):

  discounts = []
  for precos_df in df.dataCadastro:
    if precos_df is not None:
       discounts.append(np.random.random() * 0.2)
    else:
      discounts.append(0.0)

  
  return discounts


precos_df[ 'discount%' ] = add_discount_data(precos_df) 

precos_df['item_price'] = precos_df.contagem / precos_df.quantidade 

precos_df['quantidade'] = precos_df['quantidade'].astype(int)


In [ ]:
precos_df.quantidade

In [334]:

product_grain = 'produtoId_x'


sales_by_date_and_category = precos_df.groupby(['dataCadastro',product_grain]).agg(
    {
        'quantidade': 'sum',
        'item_price': np.average,
        'discount%': np.average
    }
).reset_index()


sales_by_date_and_category['day'] = pd.DatetimeIndex(sales_by_date_and_category['dataCadastro']).day
sales_by_date_and_category['month'] = pd.DatetimeIndex(sales_by_date_and_category['dataCadastro']).month

In [335]:
product_grain

'produtoId_x'

In [336]:
preco = precos_df[['produtoId_x','valorUnitario','total','valorUnitarioCusto','valorFrete','nomeunidade']]

merge_df = pd.merge(sales_by_date_and_category,preco, left_on='produtoId_x', right_on='produtoId_x')

merge_df[['dataCadastro', 'produtoId_x', 'quantidade', 'item_price', 'discount%',
       'day', 'month',  'valorUnitario', 'total',
       'valorUnitarioCusto', 'valorFrete']]


,dataCadastro,produtoId_x,quantidade,item_price,discount%,day,month,valorUnitario,total,valorUnitarioCusto,valorFrete
0,2022-01-05,1220,1,1.0,0.022031,5,1,336.74,336.74,231.41,42.25
1,2022-01-05,1265,1,1.0,0.029414,5,1,376.59,376.59,258.79,47.25
2,2022-01-05,2718,1,1.0,0.089576,5,1,396.52,396.52,272.49,49.75
3,2022-01-05,3111,2,4.0,0.051266,5,1,496.15,992.30,340.95,62.25
4,2022-01-05,3111,2,4.0,0.051266,5,1,536.01,536.01,368.34,34.86
...,...,...,...,...,...,...,...,...,...,...,...
12766,2023-03-06,14296,1,1.0,0.185413,6,3,2116.78,2116.78,1535.45,164.51
12767,2023-03-09,11027,1,1.0,0.188763,9,3,2261.28,2261.28,1640.27,238.33
12768,2023-03-09,13783,2,0.5,0.195629,9,3,707.66,1415.32,513.31,130.00
12769,2023-03-09,13784,2,0.5,0.021134,9,3,707.66,1415.32,513.31,130.00


In [ ]:
rf = RandomForestRegressor()

merge_df = merge_df.sample(frac=1)
X = merge_df[[product_grain, 'discount%', 'item_price', 'day', 'month']]
#y = merge_df['quantidade']
y = merge_df['quantidade']
rf = rf.fit(X, y)
rf

In [338]:
merge_df['predicted_quantity'] = rf.predict(X)
# Compute mean absolute error on predictions
merge_df['error'] = abs(merge_df.quantidade - merge_df.predicted_quantity)
print(f"Mean Absolute Error: {merge_df['error'].mean()}")

Mean Absolute Error: 0.010110406389476157


In [339]:
# Prever a quantidade nos dados
merge_df[ 'quantidadeprevisa' ] = rf.predict(X) 
# Calcular o erro absoluto médio nas previsões
merge_df[ 'error' ] = abs(merge_df[ 'quantidade' ] - merge_df.predicted_quantity) 

In [ ]:
sales_by_date_and_category[sales_by_date_and_category['dataCadastro'] >= '2022-01-01']
categories = sales_by_date_and_category.produtoId_x.unique()
categories


In [341]:
def optimize_quantity_out(model, df, product_grain, product_grain_value, day, month, grain=50):
    
    item_price_mean = df[df[product_grain] == product_grain_value].valorUnitario.mean()
    
  
    potential_price = np.linspace(item_price_mean * 0.25, item_price_mean * 2, grain)
 
    potential_discount = np.linspace(0, 0.7, grain)
    samples = []
    for d in potential_discount:

      for p in potential_price:
        sample = {
            product_grain: product_grain_value,
            'discount%': d,
            'item_price': p,
            'day': day,
            'month': month,
            'produtoId_x':product_grain_value
            
        }
      samples.append(sample)
 
    samples = pd.DataFrame(samples)
    samples.drop_duplicates()
    samples['Previsao'] = model.predict(samples)
    samples['is_optimal'] = [True if s == max(samples.Previsao) else False for s in samples.Previsao]
    dict_Sample = samples.to_dict('records')

    return dict_Sample



In [342]:
import random
def get_predicts(*args, **kwargs):
    sku = random.choice(categories)
    new_dict = merge_df.to_dict('records')
    lsits = [{**arg} for arg in new_dict]
    cont = len(lsits)
    i = 1
    
    while i < cont:
        predicts = {}
        if lsits[i]['produtoId_x'] == sku:

            try:
                new_predicts = optimize_quantity_out(rf, merge_df, product_grain, lsits[i]['produtoId_x']
                                                    , kwargs['day'], kwargs['month'] , grain=60)
    

                return new_predicts
                     

            except:
                pass
        
        i+=1


lista_dicts = []
datas = merge_df[['day','month','dataCadastro','valorUnitario','total','valorUnitarioCusto','valorFrete','quantidade','nomeunidade']].head(10)


datas_t = datas.to_dict('records')
for d in datas_t:
        
    new_predicts_dict = get_predicts(day=d['day'],month=d['month'],loja=d['nomeunidade'])
    dicts = [{**arg} for arg in new_predicts_dict]
    for items in dicts:
        new_dicts = {
            "dataCadastro":d['dataCadastro'],
            "valorUnitario":d['valorUnitario'],
            "total":d['total'],
            "valorUnitarioCusto":d['valorUnitarioCusto'],
            "valorFrete":d['valorFrete'],
            "loja":d['nomeunidade']
        }

        new_dicts.update(items)
        
        lista_dicts.append(new_dicts)
   

print(lista_dicts)
      


[{'dataCadastro': '2022-07-07', 'valorUnitario': 3862.58, 'total': 3862.58, 'valorUnitarioCusto': 3016.58, 'valorFrete': 0.0, 'loja': 'Maceió - Jatiúca', 'produtoId_x': 3131, 'discount%': 0.0, 'item_price': 2200.0, 'day': 7, 'month': 7, 'Previsao': 1.0, 'is_optimal': False}, {'dataCadastro': '2022-07-07', 'valorUnitario': 3862.58, 'total': 3862.58, 'valorUnitarioCusto': 3016.58, 'valorFrete': 0.0, 'loja': 'Maceió - Jatiúca', 'produtoId_x': 3131, 'discount%': 0.011864406779661016, 'item_price': 2200.0, 'day': 7, 'month': 7, 'Previsao': 1.0, 'is_optimal': False}, {'dataCadastro': '2022-07-07', 'valorUnitario': 3862.58, 'total': 3862.58, 'valorUnitarioCusto': 3016.58, 'valorFrete': 0.0, 'loja': 'Maceió - Jatiúca', 'produtoId_x': 3131, 'discount%': 0.02372881355932203, 'item_price': 2200.0, 'day': 7, 'month': 7, 'Previsao': 1.0, 'is_optimal': False}, {'dataCadastro': '2022-07-07', 'valorUnitario': 3862.58, 'total': 3862.58, 'valorUnitarioCusto': 3016.58, 'valorFrete': 0.0, 'loja': 'Maceió 

In [346]:
df_predict = pd.DataFrame(lista_dicts)

df_predict['discount%'] = (df_predict['discount%']*100).round(2)
df_predict['item_price'] = df_predict['item_price'].round(2)
df_predict['Previsao'] = df_predict['Previsao'].apply(lambda k: abs(round(k,0)))

df_predict = df_predict[(df_predict['discount%'] > 0) & (df_predict['discount%'] <70)]
df_predict = df_predict[df_predict['Previsao'] > 0]
df_predict = df_predict.drop_duplicates(subset=['produtoId_x','item_price'])
df_predict.set_index('dataCadastro')


,valorUnitario,total,valorUnitarioCusto,valorFrete,loja,produtoId_x,discount%,item_price,day,month,Previsao,is_optimal
dataCadastro,,,,,,,,,,,,
2022-07-07,3862.58,3862.58,3016.58,0.00,Maceió - Jatiúca,3131,1.19,2200.00,7,7,1.0,False
2022-03-29,4856.49,4856.49,3792.81,0.00,Fortaleza,2983,1.19,3625.68,29,3,1.0,True
2022-07-29,878.30,2634.90,379.00,105.00,Irecê,7704,1.19,4470.72,29,7,1.0,False
2022-10-17,3271.57,3271.57,2468.41,0.00,Nova Lima - Vila da Serra,9503,1.19,2202.90,17,10,1.0,False
2022-07-26,873.81,873.81,609.46,50.11,Aparecida de Goiânia - Jardim Luz,8200,1.19,1276.89,26,7,1.0,False
2022-05-25,488.97,488.97,341.04,34.35,Salvador - Barra,1210,1.19,717.76,25,5,1.0,False
2022-03-09,4856.49,4856.49,3792.81,0.00,Vitória - Praia do Canto,744,1.19,530.04,9,3,1.0,False
2022-06-24,1378.61,1378.61,1076.66,0.00,Maceió - Jatiúca,10892,1.19,2538.50,24,6,1.0,False
2022-04-25,5633.89,5633.89,4399.94,0.00,Porto Alegre - Partenon,10770,1.19,2188.30,25,4,1.0,True


In [347]:
fig = px.scatter_3d(df_predict, x='valorUnitario', y='discount%'
                    , z='Previsao',text="produtoId_x",
                      color='dataCadastro',hover_name="produtoId_x",log_x=True
                    , width=900, height=900)

fig.update_traces(textposition='top center')

fig.update_traces(mode="markers+lines")

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
      
    )
)


fig.show()

In [348]:

fig = px.line(df_predict, x="month", y="valorUnitario", color="loja",text="discount%",hover_name="produtoId_x"
              , title="DescontoPrevisto")
fig.update_traces(mode="markers+lines")

fig.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across")
fig.update_yaxes(showspikes=True, spikecolor="orange", spikethickness=2)
fig.update_layout(spikedistance=1000, hoverdistance=100)

fig.show()